# Notkun gervigreindar fyrir teikningu þrívíddarmynda

Nathan HK

In [2]:
import numpy as np
import os
import pywikibot
import sklearn as sk
from sklearn.model_selection import train_test_split
from stl import mesh
import time
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
torch.set_default_device(device)
mappa = '/Users/002-nathan/Desktop/Envalys/envalys-nathan/'  # Change this

## Inngangsorð
Við ætlum að þjálfa gervigreindarlíkan til að teikna þrívíddarmyndir. Hvernig gerum við það?
- Fyrst reyndi ég að nota þrívíddardíla eða "þríla" (e. *voxels*). Ég taldi punktana í hverjum þríl, og þjálfaði líkan með tölunum, eins og líkan sem greinir ljósmyndir. GitHub: https://github.com/nholmesking/envalys-nathan/blob/4fd89a1515c1c3623a5b8cdf8c766858c465e451/Nathan%20HK.ipynb
  - Vandamál #1: Mac-GPU getur ekki neitt með þrívíddargögnum. Ég þurfti að nota CPU, sem er of hægt.
  - Vandamál #2: Hraðinn er $O(n^3)$.
  - Vandamál #3: Það var engin klár leið til að fara frá greiningu yfir á teikningu.
- Þá reyndi ég að nota líkanið 3DShape2VecSet, sem tekur lista yfir punkta og teinkir myndir sem eru eins og listinn.

Tilvísanir:
- https://arxiv.org/pdf/2301.11445
- https://github.com/1zb/3DShape2VecSet

Ég gerði litlar breytingar á upprunaforritið, því það er eitt forritasafn (e. *module*) sem virkar ekki lengur, og forritið var skrifað fyrir Windows (og ég nota Mac).

## Gögn
Gögnin frá 3DShape2VecSet eru 650 GB samtals; þetta er of mikið fyrir tölvuna mína.

Þessi gögn eru STL-skrár frá Wikimedia Commons. Það eru fimm flokkar:
- líkamshlutar
- byggingar
- rúmfræði
- geimfarartæki
- styttur

### Sækja gögn

In [3]:
flokkar = ['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
skrar = {}
catnum = {}

In [4]:
commons = pywikibot.Site('commons', 'commons')
cn = 0
for a in flokkar:
    print(a)
    cat = pywikibot.Category(commons, 'STL files of ' + a)
    catnum[a] = cn
    cn += 1
    n = 0
    for p in cat.members(member_type=['file']):
        if n % 10 == 0:
            print(n)
        mynd = pywikibot.FilePage(p)
        try:
            tempf = open(mappa + 'STLdata/' + a + '_' + p.title()[5:], 'r')
            tempf.close()
        except FileNotFoundError:
            mynd.download(filename=mappa + 'STLdata/' + a + '_' + p.title()[5:])
        try:
            skrar[a].append(p.title()[5:])
        except KeyError:
            skrar[a] = [p.title()[5:]]
        n += 1
        if n >= 100:
            break

body parts
0
10
20
30
40
50
60
70
80
buildings
0
10
20
geometric shapes
0
10
20
30
40
objects in space
0
10
20
30
40
50
sculptures
0
10
20
30
40
50


### Setja upp gögn fyrir notkun
Við eigum að breyta gögnunum úr STL-sniði í sniðið sem 3DShape2VecSet notar. Þetta snið er með lista yfir punkta eftir hvort þeir séu innan á forminu eða utan, en gögnin frá Wikimedia Commons er ekki með það.

Með ```isInside()``` getum útreiknað hvort lína og þríhyrningur mætast, og í hvora átt. Við búum til tvær línur sem fara beint í Z-áttina frá punktnum (eina línu upp og eina línu niður), og teljum þríhyrningana sem línan mætir; ef talan er slétt, þá er punkturinn utan á forminu, og ef talan er oddatala, þá er punkturinn innan. En það tekur of langan tíma; sumar myndir eru með fleiri en milljón þríhyrninga. Þess vegna ritaði ég tvö reiknirit til að gera þetta fljótara:
- Við deilum myndinni í 32x32x32 þríla og merkjum þá sem eru með formspunkta. Fyrir hvern listapunkt: ef þríllinn punktsins er merktur sem formspunktaþríll, þá getum við ekki neitt, en ef hann er ómerktur, þá útreiknum við hvort listapunkturinn sé innan eða utan, og fyrir alla aðra listapunkta sem eru í þessum þríl, við vitum sjálfkrafa. Og við merkjum alla þríla sem eru beint í allar áttir, fram til myndarendarins eða formspunktaþríls, sem innan/utan.
- Við deilum myndinni í 32x32 eða fleiri ílát eftir X- og Y-hnitum (því við leitum að þríhyrningum í Z-áttina). Í byrjun setjum við alla þríhyrninga í þeirra ílát; sumir þríhyrningar, sem eru á línunni milli íláta, eru settir í tvö eða fleiri ílát. Fyrir hvern listapunkt eigum við bara að leita í rétta ílátinu.

Með þessum tveim reikniritum er sniðbreytingin **tvö hundruð** sinnum fljótari. Ég skrifaði sjálfur allt hérna nema ```isInside()```; sjáðu kóðann fyrir tilvísun.

In [5]:
def isInside(a, randpoint):
    if (a[0] > randpoint[0] and a[3] > randpoint[0] and a[6] > randpoint[0]):
        return 0
    if (a[0] < randpoint[0] and a[3] < randpoint[0] and a[6] < randpoint[0]):
        return 0
    if (a[1] > randpoint[1] and a[4] > randpoint[1] and a[7] > randpoint[1]):
        return 0
    if (a[1] < randpoint[1] and a[4] < randpoint[1] and a[7] < randpoint[1]):
        return 0
 
    # This part is from https://diegoinacio.github.io/creative-coding-notebooks-page/pages/ray-intersection_triangle.html
    e = np.array([0, 0, 1])     # Ray direction
    AB = a[3:6] - a[0:3]               # Oriented segment A to B
    AC = a[6:9] - a[0:3]               # Oriented segment A to C
    n = np.cross(AB, AC)     # Normal vector
    n_ = n/np.linalg.norm(n) # Normalized normal
    if not np.any(np.dot(n_, e)):
        return 0
    d = - np.dot(n_, a[0:3])
    # Finding parameter t
    t = - (np.dot(n_, randpoint) + d)/np.dot(n_, e)
    P = randpoint + t*e
    # Get the resulting vector for each vertex
    # following the construction order
    Pa = np.dot(np.cross(a[3:6] - a[0:3], P - a[0:3]), n_)
    Pb = np.dot(np.cross(a[6:9] - a[3:6], P - a[3:6]), n_)
    Pc = np.dot(np.cross(a[0:3] - a[6:9], P - a[6:9]), n_)

    if t > 0 and (Pa > 0 and Pb > 0 and Pc > 0):
        return 1
        
    e = np.array([0, 0, -1])     # Ray direction
    # Finding parameter t
    t = - (np.dot(n_, randpoint) + d)/np.dot(n_, e)
    P = randpoint + t*e
    # Get the resulting vector for each vertex
    # following the construction order
    Pa = np.dot(np.cross(a[3:6] - a[0:3], P - a[0:3]), n_)
    Pb = np.dot(np.cross(a[6:9] - a[3:6], P - a[3:6]), n_)
    Pc = np.dot(np.cross(a[0:3] - a[6:9], P - a[6:9]), n_)

    if t > 0 and (Pa > 0 and Pb > 0 and Pc > 0):
        return -1
    return 0

In [11]:
for cat in skrar:
    tts = []
    print(cat)
    byrjun = time.time()
    os.system('mkdir "' + mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '"')
    os.system('mkdir "' + mappa + '3DShape2VecSet/data/ShapeNetV2_watertight/' + cat + '"')
    for fi in skrar[cat]:
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        utskra_mesh = open(mappa + '3DShape2VecSet/data/ShapeNetV2_watertight/' + cat + '/' + fi + '.npz', 'wb')
        np.savez(utskra_mesh, points=np.array(gogn.v0, dtype=np.float32), scale=1)
        tts.append(fi)
    train, val = train_test_split(tts)
    trainlst = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/train.lst', 'w')
    print('train')
    for fi in train:
        print(fi)
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        print(gogn.points.shape[0])
        mm = ((min([a[0] for a in gogn.v0]), max([a[0] for a in gogn.v0])),
              (min([a[1] for a in gogn.v0]), max([a[1] for a in gogn.v0])),
              (min([a[2] for a in gogn.v0]), max([a[2] for a in gogn.v0])))
        num_ilat = max(32, int((gogn.points.shape[0] / 1000) ** (1/2)))
        ilat = []
        for i in range(num_ilat):
            ilat.append([])
            for j in range(num_ilat):
                ilat[i].append([])
        ilatid = lambda x, y: (max(0, min(int((x - mm[0][0]) * num_ilat / (mm[0][1] - mm[0][0])), num_ilat - 1)),
                               max(0, min(int((y - mm[1][0]) * num_ilat / (mm[1][1] - mm[1][0])), num_ilat - 1)))
        utskra = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/' + fi + '.npz', 'wb')
        vol_points = []
        vol_label = []
        scale = (min([mm[0][0], mm[1][0], mm[2][0]]), max([mm[0][1], mm[1][1], mm[2][1]]))
        rescale = lambda x: int((x - scale[0]) * 32 / (scale[1] - scale[0]))
        voxels1 = np.zeros((32, 32, 32), dtype=np.int8)
        voxels2 = np.zeros((32, 32, 32), dtype=np.int8)
        for a in gogn.points:
            minid = ilatid(min([a[0], a[3], a[6]]), min([a[1], a[4], a[7]]))
            maxid = ilatid(max([a[0], a[3], a[6]]), max([a[1], a[4], a[7]]))
            for i in range(minid[0], maxid[0] + 1):
                for j in range(minid[1], maxid[1] + 1):
                    ilat[i][j].append(a)
            for i1 in range(min([rescale(a[0]), rescale(a[3]), rescale(a[6])]), max([rescale(a[0]), rescale(a[3]), rescale(a[6])]) + 1):
                if i1 < 0:
                    i = 0
                elif i1 >= voxels1.shape[0]:
                    i = voxels1.shape[0] - 1
                else:
                    i = i1
                for i2 in range(min([rescale(a[1]), rescale(a[4]), rescale(a[7])]), max([rescale(a[1]), rescale(a[4]), rescale(a[7])]) + 1):
                    if i2 < 0:
                        j = 0
                    elif i2 >= voxels1.shape[0]:
                        j = voxels1.shape[0] - 1
                    else:
                        j = i2
                    for i3 in range(min([rescale(a[2]), rescale(a[5]), rescale(a[8])]), max([rescale(a[2]), rescale(a[5]), rescale(a[8])]) + 1):
                        if i3 < 0:
                            k = 0
                        elif i3 >= voxels1.shape[0]:
                            k = voxels1.shape[0] - 1
                        else:
                            k = i3
                        voxels1[i][j][k] = 1
            n += 1
        for i in range(4096):
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            found = False
            if rescale(randpoint[0]) < 0:
                v0 = 0
            elif rescale(randpoint[0]) >= voxels1.shape[0]:
                v0 = voxels1.shape[0] - 1
            else:
                v0 = rescale(randpoint[0])
            if rescale(randpoint[1]) < 0:
                v1 = 0
            elif rescale(randpoint[1]) >= voxels1.shape[0]:
                v1 = voxels1.shape[0] - 1
            else:
                v1 = rescale(randpoint[1])
            if rescale(randpoint[2]) < 0:
                v2 = 0
            elif rescale(randpoint[2]) >= voxels1.shape[0]:
                v2 = voxels1.shape[0] - 1
            else:
                v2 = rescale(randpoint[2])
            while not found:
                if voxels1[v0][v1][v2] != 0:
                    break
                if voxels1[v0][v1][v2] == 0 and voxels2[v0][v1][v2] != 0:
                    found = True
                    vol_points.append(randpoint)
                    vol_label.append(max(0, voxels2[v0][v1][v2]))
                    break
                if found:
                    break
                v = [v0, v1, v2]
                while v[0] < voxels1.shape[0] and voxels1[v[0]][v1][v2] == 0:
                    if voxels2[v[0]][v1][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v[0]][v1][v2]))
                        for p in range(v0, v[0]):
                            voxels2[p][v1][v2] = voxels2[v[0]][v1][v2]
                        break
                    v[0] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[0] >= 0 and voxels1[v[0]][v1][v2] == 0:
                    if voxels2[v[0]][v1][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v[0]][v1][v2]))
                        for p in range(v[0] + 1, v0 + 1):
                            voxels2[p][v1][v2] = voxels2[v[0]][v1][v2]
                        break
                    v[0] -= 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[1] < voxels1.shape[1] and voxels1[v0][v[1]][v2] == 0:
                    if voxels2[v0][v[1]][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v[1]][v2]))
                        for p in range(v1, v[1]):
                            voxels2[v0][p][v2] = voxels2[v0][v[1]][v2]
                        break
                    v[1] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[1] >= 0 and voxels1[v0][v[1]][v2] == 0:
                    if voxels2[v0][v[1]][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v[1]][v2]))
                        for p in range(v[1] + 1, v1 + 1):
                            voxels2[v0][p][v2] = voxels2[v0][v[1]][v2]
                        break
                    v[1] -= 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[2] < voxels1.shape[2] and voxels1[v0][v1][v[2]] == 0:
                    if voxels2[v0][v1][v[2]] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v1][v[2]]))
                        for p in range(v2, v[2]):
                            voxels2[v0][v1][p] = voxels2[v0][v1][v[2]]
                        break
                    v[2] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[2] >= 0 and voxels1[v0][v1][v[2]] == 0:
                    if voxels2[v0][v1][v[2]] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v1][v[2]]))
                        for p in range(v[2] + 1, v2 + 1):
                            voxels2[v0][v1][p] = voxels2[v0][v1][v[2]]
                        break
                    v[2] -= 1
                break
            if not found:
                fata = ilatid(randpoint[0], randpoint[1])
                pct = [0, 0]
                for a in ilat[fata[0]][fata[1]]:
                    isi = isInside(a, randpoint)
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
                if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                    vol_points.append(randpoint)
                    vol_label.append(pct[0] % 2)
                    if pct[0] % 2 == 1:
                        voxels2[v0][v1][v2] = 1
                    else:
                        voxels2[v0][v1][v2] = -1
        print('train vol', time.time() - byrjun)
        near_points = []
        near_label = []
        for i in range(4096):
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            fata = ilatid(randpoint[0], randpoint[1])
            pct = [0, 0]
            for a in ilat[fata[0]][fata[1]]:
                isi = isInside(a, randpoint)
                if isi == 1:
                    pct[1] += 1
                elif isi == -1:
                    pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                near_points.append(randpoint)
                near_label.append(pct[0] % 2)
        print('train near', time.time() - byrjun)
        if len(vol_points) > 0 and len(near_points) > 0:
            trainlst.write(fi + '.npz\n')
            np.savez(utskra,
                     vol_points=np.array(vol_points, dtype=np.float32),
                     vol_label=np.array(vol_label),
                     near_points=np.array(near_points, dtype=np.float32),
                     near_label=np.array(near_label))
    trainlst.close()
    vallst = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/val.lst', 'w')
    print('val')
    for fi in val:
        print(fi)
        vallst.write(fi + '.npz\n')
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        print(gogn.points.shape[0])
        mm = ((min([a[0] for a in gogn.v0]), max([a[0] for a in gogn.v0])),
              (min([a[1] for a in gogn.v0]), max([a[1] for a in gogn.v0])),
              (min([a[2] for a in gogn.v0]), max([a[2] for a in gogn.v0])))
        num_ilat = max(32, int((gogn.points.shape[0] / 1000) ** (1/2)))
        ilat = []
        for i in range(num_ilat):
            ilat.append([])
            for j in range(num_ilat):
                ilat[i].append([])
        ilatid = lambda x, y: (max(0, min(int((x - mm[0][0]) * num_ilat / (mm[0][1] - mm[0][0])), num_ilat - 1)),
                               max(0, min(int((y - mm[1][0]) * num_ilat / (mm[1][1] - mm[1][0])), num_ilat - 1)))
        utskra = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/' + fi + '.npz', 'wb')
        vol_points = []
        vol_label = []
        scale = (min([mm[0][0], mm[1][0], mm[2][0]]), max([mm[0][1], mm[1][1], mm[2][1]]))
        rescale = lambda x: int((x - scale[0]) * 32 / (scale[1] - scale[0]))
        voxels1 = np.zeros((32, 32, 32), dtype=np.int8)
        voxels2 = np.zeros((32, 32, 32), dtype=np.int8)
        for a in gogn.points:
            minid = ilatid(min([a[0], a[3], a[6]]), min([a[1], a[4], a[7]]))
            maxid = ilatid(max([a[0], a[3], a[6]]), max([a[1], a[4], a[7]]))
            for i in range(minid[0], maxid[0] + 1):
                for j in range(minid[1], maxid[1] + 1):
                    ilat[i][j].append(a)
            for i1 in range(min([rescale(a[0]), rescale(a[3]), rescale(a[6])]), max([rescale(a[0]), rescale(a[3]), rescale(a[6])]) + 1):
                if i1 < 0:
                    i = 0
                elif i1 >= voxels1.shape[0]:
                    i = voxels1.shape[0] - 1
                else:
                    i = i1
                for i2 in range(min([rescale(a[1]), rescale(a[4]), rescale(a[7])]), max([rescale(a[1]), rescale(a[4]), rescale(a[7])]) + 1):
                    if i2 < 0:
                        j = 0
                    elif i2 >= voxels1.shape[0]:
                        j = voxels1.shape[0] - 1
                    else:
                        j = i2
                    for i3 in range(min([rescale(a[2]), rescale(a[5]), rescale(a[8])]), max([rescale(a[2]), rescale(a[5]), rescale(a[8])]) + 1):
                        if i3 < 0:
                            k = 0
                        elif i3 >= voxels1.shape[0]:
                            k = voxels1.shape[0] - 1
                        else:
                            k = i3
                        voxels1[i][j][k] = 1
        for i in range(4096):
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            found = False
            if rescale(randpoint[0]) < 0:
                v0 = 0
            elif rescale(randpoint[0]) >= voxels1.shape[0]:
                v0 = voxels1.shape[0] - 1
            else:
                v0 = rescale(randpoint[0])
            if rescale(randpoint[1]) < 0:
                v1 = 0
            elif rescale(randpoint[1]) >= voxels1.shape[0]:
                v1 = voxels1.shape[0] - 1
            else:
                v1 = rescale(randpoint[1])
            if rescale(randpoint[2]) < 0:
                v2 = 0
            elif rescale(randpoint[2]) >= voxels1.shape[0]:
                v2 = voxels1.shape[0] - 1
            else:
                v2 = rescale(randpoint[2])
            while not found:
                if voxels1[v0][v1][v2] != 0:
                    break
                if voxels1[v0][v1][v2] == 0 and voxels2[v0][v1][v2] != 0:
                    found = True
                    vol_points.append(randpoint)
                    vol_label.append(max(0, voxels2[v0][v1][v2]))
                    break
                if found:
                    break
                v = [v0, v1, v2]
                while v[0] < voxels1.shape[0] and voxels1[v[0]][v1][v2] == 0:
                    if voxels2[v[0]][v1][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v[0]][v1][v2]))
                        for p in range(v0, v[0]):
                            voxels2[p][v1][v2] = voxels2[v[0]][v1][v2]
                        break
                    v[0] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[0] >= 0 and voxels1[v[0]][v1][v2] == 0:
                    if voxels2[v[0]][v1][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v[0]][v1][v2]))
                        for p in range(v[0] + 1, v0 + 1):
                            voxels2[p][v1][v2] = voxels2[v[0]][v1][v2]
                        break
                    v[0] -= 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[1] < voxels1.shape[1] and voxels1[v0][v[1]][v2] == 0:
                    if voxels2[v0][v[1]][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v[1]][v2]))
                        for p in range(v1, v[1]):
                            voxels2[v0][p][v2] = voxels2[v0][v[1]][v2]
                        break
                    v[1] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[1] >= 0 and voxels1[v0][v[1]][v2] == 0:
                    if voxels2[v0][v[1]][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v[1]][v2]))
                        for p in range(v[1] + 1, v1 + 1):
                            voxels2[v0][p][v2] = voxels2[v0][v[1]][v2]
                        break
                    v[1] -= 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[2] < voxels1.shape[2] and voxels1[v0][v1][v[2]] == 0:
                    if voxels2[v0][v1][v[2]] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v1][v[2]]))
                        for p in range(v2, v[2]):
                            voxels2[v0][v1][p] = voxels2[v0][v1][v[2]]
                        break
                    v[2] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[2] >= 0 and voxels1[v0][v1][v[2]] == 0:
                    if voxels2[v0][v1][v[2]] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v1][v[2]]))
                        for p in range(v[2] + 1, v2 + 1):
                            voxels2[v0][v1][p] = voxels2[v0][v1][v[2]]
                        break
                    v[2] -= 1
                break
            if not found:
                fata = ilatid(randpoint[0], randpoint[1])
                pct = [0, 0]
                for a in ilat[fata[0]][fata[1]]:
                    isi = isInside(a, randpoint)
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                vol_points.append(randpoint)
                vol_label.append(pct[0] % 2)
        if len(vol_points) > 0:
            print('val vol', time.time() - byrjun)
            np.savez(utskra,
                     vol_points=np.array(vol_points, dtype=np.float32),
                     vol_label=np.array(vol_label))
    vallst.close()
    print(time.time() - byrjun)
    print('----')

body parts


mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/body parts: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/body parts: File exists


train
BodyParts3D FJ6400 Manubrium.stl
2526
train vol 4.274858713150024
train near 6.624835729598999
Vh-m-eye-r.stl
346630
train vol 33.42395281791687
train near 63.87219190597534
We Are Beautiful – 453703-OSNN-1 – surface.stl
36981
train vol 66.84664177894592
train near 68.40646195411682
Orbit of the Face, STL file processed 1.0.0.stl
517986
train vol 93.1724009513855
train near 102.68889284133911
We Are Beautiful – 591522-FSAN-2 – surface.stl
308120
train vol 114.64771175384521
train near 120.16921782493591
We Are Beautiful – 591522-FSAN-1 – solid.stl
592592
train vol 142.52585291862488
train near 151.24378275871277
We Are Beautiful – 842635-FSAN-1 – solid.stl
149898
train vol 157.5586450099945
train near 160.85660481452942
We Are Beautiful – 554268-FSAN-1 – solid.stl
423450
train vol 176.69494462013245
train near 183.1288619041443
We Are Beautiful – 591522-FSAN-3 – surface.stl
130469
train vol 189.08080792427063
train near 192.3293318748474
First Rib (Human).stl
11232
train vol 193.

  n_ = n/np.linalg.norm(n) # Normalized normal



train near 862.4744818210602
We Are Beautiful – 320912-FSAN-1 – solid.stl
597660
train vol 884.5361318588257
train near 893.3718268871307
Vh-m-heart.stl
164119
train vol 904.6034078598022
train near 913.1983828544617
We Are Beautiful – 693514-FSAN-1 – surface.stl
67209
train vol 916.7990820407867
train near 919.432835817337
We Are Beautiful – 591522-FSAN-4 – solid.stl
414088
train vol 935.0758879184723
train near 942.2999258041382
We Are Beautiful – 628398-FSAN-1 – solid.stl
235046
train vol 951.7774457931519
train near 956.1246609687805
3D ear.stl
222732
train vol 966.2731008529663
train near 972.3458507061005
3d-vh-m-trachea.stl
44042
train vol 975.4637417793274
train near 979.5217337608337
Zahn 20230620 001.stl
24576
train vol 981.2402119636536
train near 983.4739367961884
Reconstrução facial - busto - Cáceres-MT.stl
199966
train vol 991.699179649353
train near 996.8788149356842
3DPX-008679 Foot Skin Nevit Dilmen.stl
467956
train vol 1014.6418209075928
train near 1022.1876187324524


...

train
Fcc bz 2.stl
220
train vol 2.816995859146118
train near 4.6203179359436035
Bcc bz 3.stl
68
train vol 7.211393117904663
train near 9.887775897979736
Bcc bz 1.stl
46
train vol 11.082934141159058
train near 12.733376026153564
LowPoly Sphere 07.stl
52
train vol 14.066622972488403
train near 15.633478879928589
Surface de Boy.stl
13923
train vol 18.20611882209778
train near 21.744298934936523
Soccer ball.stl
16380
train vol 24.384565114974976
train near 28.6063449382782
3D model of Fermat cubic.stl
8470
train vol 29.515389919281006
train near 30.76482582092285
Inverse-reuleaux-tetrahedron-2-intersection.stl
11158
train vol 32.207504987716675
train near 33.57531189918518
LowPoly Sphere 12.stl
140
train vol 34.776174783706665
train near 36.43340182304382
Earth.stl
122880
train vol 41.9093451499939
train near 45.85707712173462
Inverse-reuleaux-tetrahedron-3-intersection.stl
5786
train vol 47.15355396270752
train near 48.42105293273926
Goldberg polyhedron with equal edge lengths.stl
636
tr

mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/objects in space: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/objects in space: File exists


exception (False, 'No lines found, impossible to read')
train
Cassini 3D Model.stl
25736
train vol 4.095124006271362
train near 6.645594120025635
Juno Probe.stl
21514
train vol 8.769208192825317
train near 10.961579084396362
Galileo without probe 3D.stl
16729
train vol 12.821896076202393
train near 15.656591892242432
Aura 27.stl
87333
train vol 48.194682121276855
train near 85.94612097740173
Apollo command module smithsonian high detail.stl
3074098
train vol 192.22590112686157
train near 213.15058422088623
GeoTailSAT.stl
9416
train vol 215.76136827468872
train near 220.62798404693604
Aquarius-2010-Composite.stl
120568
train vol 234.4772961139679
train near 255.45305109024048
Curiosity Rover.stl
114899
train vol 261.3049490451813
train near 266.3058121204376
Cloudsat1-composite.stl
45313
train vol 272.90712308883667
train near 281.57013511657715
Chandra v09.stl
30132
train vol 284.0751051902771
train near 286.2589530944824
OSIRIS-REx.stl
29936
train vol 288.23839116096497
train near 290

mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/sculptures: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/sculptures: File exists


train
Bust of Medusa.stl
629736
train vol 31.783539295196533
train near 45.494203090667725
The Age Of Bronze (Auguste Rodin).stl
1880066
train vol 111.44643807411194
train near 130.86036610603333
妻野神（町田市高ヶ坂）.stl
1246474
train vol 176.3013792037964
train near 193.22359704971313
Thingiverse - Bastet.stl
16524
train vol 196.27543210983276
train near 199.2715141773224
庚申塔（十二神社）.stl
1245758
train vol 242.31276321411133
train near 256.56241726875305
聖徳太子塔（三反田稲荷社）.stl
1777819
train vol 325.3608560562134
train near 356.3638699054718
Wasserbüffel und Knabe - 3D model by noe-3d.at - Sketchfab.stl
661492
train vol 380.2654540538788
train near 390.6559510231018
庚申塔（寺山杉山神社）.stl
1200000
train vol 436.9429261684418
train near 453.0380711555481
五神名地神塔（横浜市旭区）.stl
1473158
train vol 514.1006619930267
train near 556.4925272464752
Scan the World - Juno Ludovisi.stl
642984
train vol 581.3900411128998
train near 593.615195274353
庚申塔道標（横浜市旭区今宿南町）.stl
1750000
train vol 652.9628012180328
train near 668.25958919

Sniðbreytingin tók nærri 4 klst.

## Líkan
Líkanið þarf ekki að búa til góðar myndir. Það þarf bara að virka án villna, sem sýnir okkur að við þurfum bara meiri og betri gögn, því vísindagreinin er með góðar myndir.

In [18]:
os.system('cd ' + mappa)
os.system('torchrun --nproc_per_node=4 3DShape2VecSet/main_ae.py --accum_iter=2 --model ae_d512_m512 '
          '--data_path 3DShape2VecSet/data --output_dir output/ae/ae_d512_m512 --log_dir output/ae/ae_d512_m512 '
          '--num_workers 10 --point_cloud_size 2048 --batch_size 64 --epochs 200 --warmup_epochs 5')

W0825 09:05:21.680789 8286703424 torch/distributed/elastic/multiprocessing/redirects.py:28] NOTE: Redirects are currently not supported in Windows or MacOs.
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] 
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] *****************************************
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] *****************************************


['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']

['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
Model = AutoEncoder(
  (cross_attend_blocks): ModuleList(
    (0): PreNorm(
      (fn): Attention(
        (to_q): Linear(in_features=512, out_features=512, bias=False)
        (to_kv): Linear(in_features=512, out_features=1024, bias=False)
        (to_out): Linear(in_features=512, out_features=512, bias=True)
        (drop_path): Identity()


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Epoch: [0]  [ 0/47]  eta: 0:43:45  lr: 0.000000  loss: 0.7250 (0.7250)  loss_vol: 0.6592 (0.6592)  loss_near: 0.6580 (0.6580)  iou: 0.0139 (0.0139)  time: 55.8586  data: 19.8939
Epoch: [0]  [ 0/47]  eta: 0:49:22  lr: 0.000000  loss: 0.7185 (0.7185)  loss_vol: 0.6532 (0.6532)  loss_near: 0.6534 (0.6534)  iou: 0.0089 (0.0089)  time: 63.0310  data: 19.9302
Epoch: [0]  [ 0/47]  eta: 0:50:54  lr: 0.000000  loss: 0.7204 (0.7204)  loss_vol: 0.6549 (0.6549)  loss_near: 0.6550 (0.6550)  iou: 0.0234 (0.0234)  time: 64.9863  data: 16.6555
Epoch: [0]  [ 0/47]  eta: 1:23:32  lr: 0.000000  loss: 0.7272 (0.7272)  loss_vol: 0.6611 (0.6611)  loss_near: 0.6610 (0.6610)  iou: 0.0238 (0.0238)  time: 106.6386  data: 16.7561


W0825 09:35:12.375863 8286703424 torch/distributed/elastic/agent/server/api.py:688] Received Signals.SIGINT death signal, shutting down workers
W0825 09:35:12.392325 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40348 closing signal SIGINT
W0825 09:35:12.393383 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40349 closing signal SIGINT
W0825 09:35:12.393764 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40350 closing signal SIGINT
W0825 09:35:12.398679 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40351 closing signal SIGINT
W0825 09:35:42.408644 8286703424 torch/distributed/elastic/multiprocessing/api.py:875] Unable to shutdown process 40348 via Signals.SIGINT, forcefully exiting via Signals.SIGKILL
W0825 09:35:49.303166 8286703424 torch/distributed/elastic/multiprocessing/api.py:875] Unable to shutdown process 40349 via Signals.SIGINT, forcefully ex

256

Það eru 200 lotna (e. *epochs*), og hver lota tekur klukkustund; við eigum ekki nógan tíma fyrir alla þjálfunina. **En hún er hafin án villna.**

# Lokaorð
Hvert förum við héðan? 3DShape2VecSet er með margar takmarkanir.
- Það kann bara að búa til einfaldar myndir, eins og "stóll". Með greiningarlíkani getum við mögulega búið til fjölfaldar myndir. Ef notandi segir "stóll með kodda", þá býr líkanið okkar til mynd af stóli og mynd af kodda, sem líkanið sameinar.
- Það skilur ekki mannamál.
- Það er ekki með mikil gögn. Með mínu forriti hérna getum við búið til eins mikil gögn og við viljum; við getum tekið myndir frá notendum og búum til lista yfir punkta, og notað þá til að þjálfa líkanið.

Ég er viss um að, með meiri gögn og betri tölvu, við getum að minnsta kosti eitt af þessum.